Apparently the cluster one is not working and I dont know why, I thoguht it was only because of the bkg, but apparently not, so i am running the fit on jupyter notebook

# fitting in jupyternotebook

In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
import matplotlib.pyplot as plt
import numpy as np
from regions import CircleSkyRegion
import yaml

from gammapy.catalog import CATALOG_REGISTRY
from gammapy.data import EventList
from gammapy.datasets import MapDataset
from gammapy.irf import PSFMap, EDispKernelMap
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
    TemplateSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SuperExpCutoffPowerLaw4FGLSpectralModel,
    create_fermi_isotropic_diffuse_model,
    ExpCutoffPowerLawSpectralModel,
    TemplateSpectralModel,
)
from gammapy.modeling import Fit

##################################
# parameters I can change

dataset_idx = 1
n_bin_met = 0
thresh_idx = 20
###################################
# general definitions

dataset_names = ['low_energy', 'med_energy', 'hi_energy']
main_path = '/home/woody/caph/mppi062h/woody_output/final/fermilat/'
iso_list = ['iso_P8R3_ULTRACLEANVETO_V3_v1.txt', 'iso_P8R3_ULTRACLEANVETO_V3_v1.txt', 'iso_P8R3_SOURCE_V3_v1.txt']

N_bin_met = 8
radius_list = [12, 5, 5]
dataset_e_edges = [100, 600, 4e3, 1e6] #u.MeV

with open('/home/vault/caph/mppi062h/repositories/GC/FermiLAT/source_list.txt') as f:
    source_list = f.readlines()
    
fgl = CATALOG_REGISTRY.get_cls("4fgl")()  

### V1: same model, for both low and high energy dataset

Here the whole energy range, which includes 2 dataets will share the same input model for GC, so it is a joint fit

In [2]:
from gammapy.datasets import Datasets

def fit_all(n_bin_met, thresh_idx=20):
    # opening the specific dataset
    dataset_list = []
    for dataset_idx in [1,2]:
        folder = f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met'
        dataset = MapDataset.read(f'{folder}/fermi_dataset.fits', name=f'dataset_idx={dataset_idx}')
        dataset.background = Map.from_geom(geom=dataset.counts.geom, data=np.zeros_like(dataset.counts.data))   

        dataset.mask_safe = Map.from_geom(geom=dataset.counts.geom, data=np.ones_like(dataset.counts.data).astype(bool))   
        dataset.mask_safe &= dataset.counts.geom.region_mask(f"galactic;circle(0, 0, {radius_list[dataset_idx]})")

        m1 = (dataset.counts.geom.axes['energy'].center.value > dataset_e_edges[dataset_idx])
        m2 = (dataset.counts.geom.axes['energy'].center.value < dataset_e_edges[dataset_idx+1])
        mask = m1 & m2
        dataset.mask_safe.data[~mask] = 0

        # opening the source maps, done before
        for source_name in source_list[:thresh_idx]:
            if source_name[:-1] != '4FGL J1745.6-2859':
                temp = MapDataset.read(f'{folder}/sources_dataset/{source_name[5:-1]}.fits')
                dataset.background.data += temp.counts.data

        dataset_list.append(dataset)
    datasets = Datasets(dataset_list)
    datasets.name=f'joint_{n_bin_met}'

    ### setting the models for this dataset, only central source the the diffuse
    fermi_models = []
    mo = fgl['4FGL J1745.6-2859'].sky_model()    
    mo.spatial_model.parameters.freeze_all()
    fermi_models.append(mo)
    initial_model = fermi_models

    filename='/home/vault/caph/mppi062h/repositories/GC/FermiLAT/gll_iem_v07.fits'
    diffuse_galactic_fermi = Map.read(filename)
    diffuse_galactic_fermi.unit = "cm-2 s-1 MeV-1 sr-1"
    template_diffuse = TemplateSpatialModel(diffuse_galactic_fermi, normalize=False, filename=filename)

    diffuse_iem = SkyModel(
        spectral_model=PowerLawNormSpectralModel(),
        spatial_model=template_diffuse,
        name="diffuse-iem",
    )
    diffuse_iem.parameters['tilt'].frozen = False
    initial_model.append(diffuse_iem)

    for dataset_idx in [1,2]:
        filename = f'{main_path}/{dataset_names[dataset_idx]}/{iso_list[dataset_idx]}'
        diffuse_iso = create_fermi_isotropic_diffuse_model(filename=filename, interp_kwargs={"fill_value": None})
        diffuse_iso.datasets_names = dataset_list[dataset_idx-1].name
        diffuse_iso.parameters['norm'].min=0
        initial_model.append(diffuse_iso.copy(name=f'fermi-diffuse-iso_{dataset_idx}'))
    datasets.models = initial_model   

    print(datasets.models)
    print(dataset_list[dataset_idx-1].name)
    ########## doing the fit
    result0= Fit().run(datasets=[datasets])
    print(result0)
    print(datasets.models)

    for dataset_idx in [1,2]:
        with open(f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met/fitted_model_joint.yml', 'w') as outfile:
            yaml.dump(datasets.models.to_dict(), outfile, default_flow_style=False)

In [ ]:
for n_bin_met in range(N_bin_met-1):
    fit_all(n_bin_met, thresh_idx=20)
    print(n_bin_met)

DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   2.75e-12   +/- 5.8e-14 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.565   +/-    0.02             
    beta                    :      0.170   +/-    0.01             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         

Component 1: SkyModel

  Name                      : diffuse-iem
  Datasets names            : None
  Spectral model type       : PowerLawNormSpectralModel
  Spatial  model type       : TemplateSpatialModel
  Temporal model type       : 
  Parameters:
    norm                    :      1.000   +/-    0.00             
    tilt               

Template file already exits, and overwrite is False
Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 366
	total stat : 93461.53

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 366
	total stat : 93461.53


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   3.02e-12   +/- 2.2e-13 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.530   +/-    0.09             
    beta                    :      0.257   +/-    0.06             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False
Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 391
	total stat : 93302.24

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 391
	total stat : 93302.24


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   2.58e-12   +/- 2.1e-13 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.661   +/-    0.18             
    beta                    :      0.295   +/-    0.12             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False
Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 376
	total stat : 81296.75

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 376
	total stat : 81296.75


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   2.25e-12   +/- 1.9e-13 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.948   +/-    0.16             
    beta                    :      0.374   +/-    0.09             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



### V0: each dataset with its own model

Here each dataset will have a its own model for GC , so it is like independent

In [1]:
def fit_all(dataset_idx, n_bin_met, thresh_idx=20):
    # opening the specific dataset
    folder = f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met'
    dataset = MapDataset.read(f'{folder}/fermi_dataset.fits')
    dataset.background = Map.from_geom(geom=dataset.counts.geom, data=np.zeros_like(dataset.counts.data))   

    dataset.mask_safe = Map.from_geom(geom=dataset.counts.geom, data=np.ones_like(dataset.counts.data).astype(bool))   
    dataset.mask_safe &= dataset.counts.geom.region_mask(f"galactic;circle(0, 0, {radius_list[dataset_idx]})")

    m1 = (dataset.counts.geom.axes['energy'].center.value > dataset_e_edges[dataset_idx])
    m2 = (dataset.counts.geom.axes['energy'].center.value < dataset_e_edges[dataset_idx+1])
    mask = m1 & m2
    dataset.mask_safe.data[~mask] = 0

    ### setting the models for this dataset, only central source the the diffuse
    fermi_models = []
    mo = fgl['4FGL J1745.6-2859'].sky_model()    
    mo.spatial_model.parameters.freeze_all()
    fermi_models.append(mo)
    initial_model = fermi_models

    filename='/home/vault/caph/mppi062h/repositories/GC/FermiLAT/gll_iem_v07.fits'
    diffuse_galactic_fermi = Map.read(filename)
    diffuse_galactic_fermi.unit = "cm-2 s-1 MeV-1 sr-1"
    template_diffuse = TemplateSpatialModel(diffuse_galactic_fermi, normalize=False, filename=filename)

    diffuse_iem = SkyModel(
        spectral_model=PowerLawNormSpectralModel(),
        spatial_model=template_diffuse,
        name="diffuse-iem",
    )
    initial_model.append(diffuse_iem)

    filename = f'{main_path}/{dataset_names[dataset_idx]}/{iso_list[dataset_idx]}'
    diffuse_iso = create_fermi_isotropic_diffuse_model(
    filename=filename, interp_kwargs={"fill_value": None})
    initial_model.append(diffuse_iso)
    dataset.models = initial_model    

    # opening the source maps, done before
    for source_name in source_list[:thresh_idx]:
        if source_name[:-1] != '4FGL J1745.6-2859':
            temp = MapDataset.read(f'{folder}/sources_dataset/{source_name[5:-1]}.fits')
            dataset.background.data += temp.counts.data

    ########## doing the fit
    result0= Fit().run(datasets=[dataset])
    print(result0)
    print(dataset.models)

    with open(f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met/fitted_model.yml', 'w') as outfile:
        yaml.dump(dataset.models.to_dict(), outfile, default_flow_style=False)


In [3]:
for dataset_idx, dataset_name in enumerate(dataset_names):
    if dataset_idx !=0:
        for n_bin_met in range(N_bin_met-1):
            fit_all(dataset_idx, n_bin_met, thresh_idx=20)
            print(dataset_idx, n_bin_met)

Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 198
	total stat : 72543.84

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 198
	total stat : 72543.84


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   4.48e-12   +/- 1.1e-12 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      1.580   +/-    0.59             
    beta                    :     -0.171   +/-    0.28             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 185
	total stat : 73455.15

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 185
	total stat : 73455.15


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   3.42e-12   +/- 1.2e-12 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.095   +/-    0.83             
    beta                    :      0.070   +/-    0.39             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False


...

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 178
	total stat : 73047.42

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 178
	total stat : 73047.42


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   3.13e-12   +/- 1.3e-12 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.338   +/-    0.94             
    beta                    :      0.139   +/-    0.43             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 216
	total stat : 71725.73

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization terminated successfully.
	nfev       : 216
	total stat : 71725.73


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   2.93e-12   +/- 5.1e-12 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      2.407   +/-    4.01             
    beta                    :      0.227   +/-    1.86             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         



Template file already exits, and overwrite is False


OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization failed.
	nfev       : 191
	total stat : 69368.93

OptimizeResult

	backend    : minuit
	method     : migrad
	success    : True
	message    : Optimization failed.
	nfev       : 191
	total stat : 69368.93


DatasetModels

Component 0: SkyModel

  Name                      : 4FGL J1745.6-2859
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : PointSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude               :   2.25e-12   +/- 9.4e-13 1 / (cm2 MeV s)
    reference    (frozen)   :   3975.235       MeV         
    alpha                   :      3.091   +/-    0.88             
    beta                    :      0.543   +/-    0.40             
    lon_0        (frozen)   :    266.415       deg         
    lat_0        (frozen)   :    -28.997       deg         

Component 1: SkyModel

  Name     